In [3]:
import pandas as pd
from string import ascii_uppercase

In [4]:
with open('../data/seq_temp_binary.txt') as f:
    seq_temp = eval(f.read())

In [5]:
seq_temp_bin = [[[1 if exam > 0 else 0 for exam in event] for event in events] for events in seq_temp]

In [6]:
perfect_seq = None
with open('../data/perfect_student.txt') as f:
    perfect_seq = eval(f.read())

### Jaccard similarity between student exams temporal sequence and perfect exams order

In [7]:
def similarity(event1, event2):
    result = [0,0,0,0]
    for i in range(len(event1)):
        event_x, event_y = event1[i], event2[i]
        if event_x == 0 == event_y:
            result[0] += 1
        elif event_x == 0 and event_y == 1:
            result[1] += 1
        elif event_x == 1 and event_y == 0:
            result[2] += 1
        elif event_x == 1 == event_y:
            result[3] += 1
        else:
            raise Exception("events are not composed by only 0 and 1")
    return result
        
def jaccard(events1, events2):
    length = min(len(events1), len(events2))
    for i in range(length):
        event1, event2 = events1[i], events2[i]
        m00, m01, m10, m11 = similarity(event1, event2)
        yield m11 / (m01 + m10 + m11)

def jaccard_with_perfect_student(seq_temp_bin):
    for i, events in enumerate(seq_temp_bin):
        jaccard_ = list(jaccard(events, perfect_seq))
        jaccard_avg = sum(jaccard_) / len(jaccard_)
        yield jaccard_avg
        

- Jaccard similarity for student 1: 0.65
- Jaccard similarity for student 2: 0.225
- Jaccard similarity for student 3: 0.15000000000000002
- Jaccard similarity for student 4: 0.2555555555555556
- Jaccard similarity for student 5: 0.1
- Jaccard similarity for student 6: 0.32222222222222224
- Jaccard similarity for student 7: 0.375
- Jaccard similarity for student 8: 0.5888888888888889
- Jaccard similarity for student 9: 0.4666666666666666
- Jaccard similarity for student 10: 0.4642857142857143
- Jaccard similarity for student 11: 0.8888888888888888
- Jaccard similarity for student 12: 0.8888888888888888
- Jaccard similarity for student 13: 0.3666666666666667
- Jaccard similarity for student 14: 0.6666666666666666
- Jaccard similarity for student 15: 0.19999999999999998
- Jaccard similarity for student 16: 0.33888888888888885
- Jaccard similarity for student 17: 0.4619047619047619
- Jaccard similarity for student 18: 0.8055555555555555
- Jaccard similarity for student 19: 0.6666666666666666
- Jaccard similarity for student 20: 0.3111111111111111
- Jaccard similarity for student 21: 0.6666666666666666
- Jaccard similarity for student 22: 0.2833333333333333
- Jaccard similarity for student 23: 0.6222222222222222
- Jaccard similarity for student 24: 0.38095238095238093
- Jaccard similarity for student 25: 0.5611111111111112
- Jaccard similarity for student 26: 0.8888888888888888

In [8]:
jaccard_seq_temp = list(jaccard_with_perfect_student(seq_temp_bin))

In [9]:
cluster_avg_table = pd.read_csv('../cluster_avg_table.csv')

In [10]:
avg_per_students = list(cluster_avg_table['grade_weighted_avg'])

In [11]:
def normalize_vote(vote):
    return (vote - 18) / (31 - 18)
avg_per_students_norm = list(map(normalize_vote, avg_per_students))

In [12]:
from scipy.stats.stats import pearsonr
pearsoncoeff, pvalue = pearsonr(avg_per_students_norm, jaccard_seq_temp)
print("Pearson coefficient between student score average and jaccard similarity (see above) : ", pearsoncoeff)

Pearson coefficient between student score average and jaccard coefficient (see above):  0.499891826592


In [30]:
jaccard_avg_scores_df = \
    pd.DataFrame({"Id_studente": list(ascii_uppercase), 
                  "Jaccard similarity": jaccard_seq_temp,
                  "Avg student scores": avg_per_students_norm})\
        .reindex(columns=["Id_studente", "Jaccard similarity", "Avg student scores"])

In [31]:
jaccard_avg_scores_df.head()

Id_studente  Jaccard similarity  Avg student scores
0           A            0.650000            0.653846
1           B            0.225000            0.348562
2           C            0.150000            0.528431
3           D            0.255556            0.391608
4           E            0.100000            0.730769

In [33]:
jaccard_avg_scores_df.to_csv('../data/student_jaccard_avg_scores.csv', index=False, sep=';')